In [ ]:
import oracledb
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
from config import *
from db_conn import *

def get_NIDB_connection(conf_file):
    dsn = conf_file['NIDB_SRC']['dsn']
    user = conf_file['NIDB_SRC']['user']
    password = conf_file['NIDB_SRC']['password']
    con = oracledb.connect(user=user,password=password,dsn=dsn)
    return con

def generate_NIDB_tables(conf_file, conn):
    nidb_conn = get_NIDB_connection(conf_file)
    cur = nidb_conn.cursor()
    
    cur.execute('SELECT * FROM NSD.CRISPI_PUBS')
    crispi_pubs_columns = cur.description
    crispi_pubs_data = cur.fetchall()

    crispi_pubs_combined = list()
    for entry in crispi_pubs_data:
        temp = dict()
        for a,b in zip(crispi_pubs_columns, entry):
            temp[a[0]] = b

        crispi_pubs_combined.append(temp)

    crispi_pubs_table = pd.DataFrame(crispi_pubs_combined)

    crispi_pubs_table.insert(0,'TimeStamp',pd.to_datetime('now').replace(microsecond=0))

    crispi_pubs_table.to_sql('NIDBCrispiPubs', con=conn, if_exists='replace', index=True, index_label='id')

    cur.execute('SELECT * FROM NSD.CRISPI_REPORTS')
    crispi_reports_columns = cur.description
    crispi_reports_data = cur.fetchall()

    crispi_reports_combined = list()
    for entry in crispi_reports_data:
        temp = dict()
        for a,b in zip(crispi_reports_columns, entry):
            temp[a[0]] = b
            if a[0] == 'RPID':
                b = b.replace(' ','').replace('-','')
                b = b[1:]
                temp['RPID_FORMATTED'] = b

        crispi_reports_combined.append(temp)

    crispi_reports_table = pd.DataFrame(crispi_reports_combined)

    crispi_reports_table.insert(0,'TimeStamp',pd.to_datetime('now').replace(microsecond=0))

    crispi_reports_table.to_sql('NIDBCrispiReports', con=conn, if_exists='replace', index=True, index_label='id')


if __name__ == '__main__': 
    conf_file = load_config_file()
      
    conn = get_db_connection(conf_file)

    generate_NIDB_tables(conf_file, conn)



In [12]:
crispi_reports_table

,TimeStamp,FNAME,LNAME,ORCID,NEDID,ROLE,PRIMARY,IPID,RPID,IC,RYEAR
0,2025-01-15 11:51:30,Ruth,Lunn,0000-0002-9605-7137,0010401988,LI,N,115502,1 ZIC ES103317-05,NIEHS,2020
1,2025-01-15 11:51:30,Ruth,Lunn,0000-0002-9605-7137,0010401988,LI,Y,121300,1 ZIC ES103317-06,NIEHS,2021
2,2025-01-15 11:51:30,Ruth,Lunn,0000-0002-9605-7137,0010401988,LI,N,128028,1 ZIA ES103375-01,NIEHS,2022
3,2025-01-15 11:51:30,Ruth,Lunn,0000-0002-9605-7137,0010401988,LI,N,131200,1 ZIA ES103375-02,NIEHS,2023
4,2025-01-15 11:51:30,Ruth,Lunn,0000-0002-9605-7137,0010401988,LI,Y,136817,1 ZIA ES103375-03,NIEHS,2024
...,...,...,...,...,...,...,...,...,...,...,...
28737,2025-01-15 11:51:30,Maria,Dufau,0000-0002-3989-7113,0010038900,PI,Y,110013,1 ZIA HD000150-44,NICHD,2019
28738,2025-01-15 11:51:30,Maria,Dufau,0000-0002-3989-7113,0010038900,PI,Y,114861,1 ZIA HD000150-45,NICHD,2020
28739,2025-01-15 11:51:30,Maria,Dufau,0000-0002-3989-7113,0010038900,PI,Y,120678,1 ZIA HD000150-46,NICHD,2021
28740,2025-01-15 11:51:30,Maria,Dufau,0000-0002-3989-7113,0010038900,PI,Y,126593,1 ZIA HD000150-47,NICHD,2022
